
# Introduction Ridge VARX

In the following section the theory underlying the package is outlined. Notation is  similar to that in [Lütkepohl (2005)](http://localhost:8888/notebooks/code_example.ipynb#Bibliography) which was referenced to obtain estimators for the VAR model.

## VARX introduction

In real world settings the explicit goal is often to forecast a single time series, using a set of explanatory variables. The necessity to forecast multiple time series using the same methodology can still arise however, if future values of some explanatory variables are not available at the time of forecasting. 

The VAR formulation of the problem lets us produce forecasts of several time series simultaneously.This is achieved by putting all variables of interest in a vector. This vector is then forecasted, using past values of the vector. Hence, 'Vector autoregressive'. The remaining X in the VARX abbreviation stands for 'exogenous'.

Formally we predict a $(k\times1)$ column vector containing the values of $k$ endogenous time series at time t.

$$
y_t = Bz_t + u_t
$$


Where $B$ is a matrix of coefficients and $X_t$ is a vector containing the values of each regressor at time $t$. 

In this context, we define variables that are predicted in the vector of the VARX model as endogenous. And variables that are not influenced by (historical) values of the vector, but are in $X_t$ as exogenous. 

We can rewrite this problem, by adding columns for each value of $t\in\{1,2,...,T\}$.  

$$
Y = BZ + U
$$

And then we can solve the values of the parameters using the multivariate least squares (MLS) solution as in for instance Lutkepohl.   

$$
\hat{B} = YZ^T(ZZ^T)^{-1}
$$


## Ridge introduction

In short samples (if $T$ is small relative to $K$ where $K$ is the total number of parameters in $B$). Ridge regularization can be used to decrease the parameter estimation variance, at the cost of producing a small bias.

$$
\hat{B} = YZ^T(ZZ^T + \lambda I)^{-1}
$$


These estimates will in general improve out of sample forecasting accuracy in such a case, provided that the regularization parameter is properly tuned. Furthermore the Least Squares criterion with a ridge penalty is well posed (has an analytical solution) even in settings where there are more explanatory variables than observations.


## Estimation
In their original form both MLS and Ridge require a matrix inversion. Matrix inversions are generally avoided, due to concerns regarding machine precision for large matrices. Therefore, both models are fitted by rewriting the closed form solution and using a numerical solver. To have control over the numerical optimization the scipy.sparse.linalg.cg solver was used. This solver requires $x$ and $b$ to be vectors, in solving $Ax = b$. Our problem becomes:

$$
\underbrace{I_k\otimes (Z Z^T + \lambda I)}_\text{A} \;  \underbrace{\text{vec}  (B^T)}_\text{x} = \underbrace{\text{vec}(Z Y^T)}_\text{b} 
$$

For the positive definite and symmetric matrix $A$, the conjugate gradient method provides an efficient solution to the problem with near double precision accuracy for the parameter estimates.

## p-Values of the package

To get p-values for the estimated parameters, we need to estimate the variance of $\hat{B}(\lambda)$. The variance of the ridge estimator in our case can be found by taking the MLS estimator:

$$\hat{B} = YZ^T(ZZ^T)^{-1}$$ which has variance equal to $(ZZ^T)^{-1} \otimes \Sigma_U$, given a sample of $T$ longitudinal observations.    

And noting that the Ridge estimator $\hat{B}(\lambda)$ is equal to $\hat{B}W_{\lambda}^T$ when 

$$W_{\lambda} = (ZZ^T + \lambda I)^{-1}ZZ^T$$

And therefore the variance can easily be seen to be:

$$\text{var}(B(\lambda))= W_{\lambda} (ZZ^T)^{-1} W_{\lambda}^T \otimes \Sigma_U$$

Finally we can replace $\Sigma_U$ by its consistent in sample estimator. In our case $\Sigma_U$ will be a diagonal matrix under the assumption of correct dynamic model specification. And an unbiased in sample estimator can be found to be:

$$\frac{T}{T - (1 + pk + m)}UU^T$$

The degrees of freedom in the LS fitted model are trivially seen to be equal to the amount of fitted parameters per equation $(1 + pk + m)$. In the ridge case however, the variability of the predictions decreases as the amount of regularization increases. An estimator for the degrees of freedom in the ridge case $\widehat{\text{df}}(\lambda)$ needs to be used.

If the number of parameters per equation is bigger than $T$ the Woodburry identity provides us with an equivalent formulation of $(ZZ^T + \lambda I)^{-1}$ that requires a smaller matrix inversion and is therefore preferred.  

## Degrees of freedom ridge

We can get an estimate of the degrees of freedom by first finding the matrix $H(\lambda)$ for which: $ZB = M(\lambda)Y$ and then taking the trace of H.

In our case we find:

$$H(\lambda) = Z^T(ZZ^T + \lambda I)^{-1}Z$$


Once we have this matrix we can compute:

$$\widehat{\text{df}}(\lambda) = \text{tr}(H(\lambda)) $$


Also note that here we have $\widehat{\text{df}}(\lambda) = \text{tr}(H(\lambda))$ which is not affected by the columns in $Y$. Therefore, we will get the same result regardless of the equation that we are considering. And we can use $\widehat{\text{df}}(\lambda)$ as a scalar value in:

$$\widehat{\Sigma}_U(\lambda) = \frac{T}{T - \widehat{\text{df}}(\lambda)}UU^T$$

# Ridge parameter tuning

Finding a good value for $\lambda$ is not trivial. What makes a good value depends on the sample and can only be determined after setting an objective standard for when our model is 'good'. One way of doing this is by looking for the model that minimizes Akaike's information criterion.

For the likelihood of the entire sample we use a multivariate normal distribution under the assumption of joint normality of the sample. 

In our case we can then minimize AIC for the $h=1$ period ahead forecast by minimizing:

$$\ln(|\tilde{\Sigma}_U(\lambda)|) + \frac{2(\widehat{\text{df}}(\lambda))K}{T}$$

Where $\tilde{\Sigma}_U(\lambda)$ is the unadjusted ML estimator of the in sample error variance for any given value of $\lambda$.


# Bibliography

Lütkepohl, H. (2005). *New introduction to multiple time series analysis*. Springer Science & Business Media.